In [6]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [8]:
train_data = pd.read_csv('./data/splited_train.csv')
test_data = pd.read_csv('./data/splited_test.csv')


In [9]:
train_input = list(train_data['question_text'])
train_label = list(train_data['target'])

test_input = list(test_data['question_text'])

In [10]:

#remove all the stop words for the 
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop=stopwords.words('english') 

def remove_stop_words(x):
    for word in stop:
        token = " " + word + " "
        if (x.find(token) != -1):
            x = x.replace(token, " ")
    return x

# train_input_rsw = list(map(remove_stop_words, train_input))
# test_input_rsw = list(map(remove_stop_words, test_input))
train_input_rsw=train_input
test_input_rsw =test_input

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [11]:
max_features=100000
embed_size = 300 
max_length = 60 

In [12]:
def get_coefs(word,*arr): 
    return word, np.asarray(arr, dtype='float32')

embeddings_index = {}
with open("./data/embeddings/glove.840B.300d/glove.840B.300d.txt",'r') as f:
    for line in f:
        
        word,coefs=get_coefs(*line.split(" "))
        #coefs = np.asarray(coefs, dtype='float32')
        embeddings_index[word] = coefs

In [13]:
tokenizer=Tokenizer(num_words=max_features)

tokenizer.fit_on_texts(train_input_rsw)

word_index = tokenizer.word_index
n_words=min(max_features,len(word_index))

embedding_matrix = np.zeros((n_words+1, embed_size))

for word, i in word_index.items():
    if i >= max_features: 
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: 
        embedding_matrix[i] = embedding_vector

In [14]:
sequences = tokenizer.texts_to_sequences(train_input_rsw)

train_input_padded = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')
print(train_input_padded.shape)

sequences = tokenizer.texts_to_sequences(test_input_rsw)
test_input_padded = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')
print(test_input_padded.shape)

(1044897, 60)
(261225, 60)


In [15]:
train_text, cv_text, train_target, cv_target = train_test_split(train_input_padded, train_label, test_size = 0.1, random_state=2)

In [16]:
from keras.models import Sequential
from keras.layers import Embedding,Bidirectional,LSTM,Dropout,Conv1D,MaxPooling1D,Dense

In [17]:
#Build up a sequential model to do BiLSTM

lstm=Sequential()
lstm.add(Embedding(n_words+1,embed_size,input_length=max_length,weights=[embedding_matrix], trainable=False))
lstm.add(Bidirectional(LSTM(256,return_sequences=True)))
lstm.add(Dropout(0.2))
lstm.add(Conv1D(100,5,activation='relu'))
lstm.add(MaxPooling1D(pool_size=4))
lstm.add(LSTM(128))
lstm.add(Dropout(0.4))
lstm.add(Dense(1,activation='sigmoid'))

lstm.summary()

lstm.compile(loss='binary_crossentropy',optimizer='adam', metrics=['acc'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 60, 300)           30000300  
_________________________________________________________________
bidirectional (Bidirectional (None, 60, 512)           1140736   
_________________________________________________________________
dropout (Dropout)            (None, 60, 512)           0         
_________________________________________________________________
conv1d (Conv1D)              (None, 56, 100)           256100    
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 14, 100)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               117248    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0

In [18]:
# del embeddings_index
# import gc
# gc.collect()

In [19]:
history=lstm.fit(np.array(train_text), np.array(train_target), epochs = 5, validation_data=(np.array(cv_text),np.array(cv_target)), batch_size=1024,verbose=1)

Epoch 1/5
919/919 [==============================] - 165s 141ms/step - loss: 0.1663 - acc: 0.9418 - val_loss: 0.1164 - val_acc: 0.9558
Epoch 2/5
919/919 [==============================] - 129s 140ms/step - loss: 0.1098 - acc: 0.9568 - val_loss: 0.1059 - val_acc: 0.9584
Epoch 3/5
919/919 [==============================] - 128s 140ms/step - loss: 0.1035 - acc: 0.9592 - val_loss: 0.1041 - val_acc: 0.9592
Epoch 4/5
919/919 [==============================] - 130s 141ms/step - loss: 0.0992 - acc: 0.9607 - val_loss: 0.1005 - val_acc: 0.9603
Epoch 5/5
919/919 [==============================] - 128s 140ms/step - loss: 0.0946 - acc: 0.9621 - val_loss: 0.1004 - val_acc: 0.9597


In [20]:
from sklearn.metrics import f1_score
cv_predictions = lstm.predict(cv_text, batch_size=512)

thresholds = []
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    result = f1_score(cv_target, (cv_predictions>thresh).astype(int))
    thresholds.append([thresh, result])
    print("F1 score at threshold {} is {}".format(thresh, result))

thresholds.sort(key=lambda x: x[1], reverse=True)
print("Best value {}".format(thresholds[0]))
best_thresh = thresholds[0]

F1 score at threshold 0.1 is 0.6084311632870865
F1 score at threshold 0.11 is 0.6169587460542071
F1 score at threshold 0.12 is 0.623192420632722
F1 score at threshold 0.13 is 0.628404231375197
F1 score at threshold 0.14 is 0.6337706788362807
F1 score at threshold 0.15 is 0.6382461823229987
F1 score at threshold 0.16 is 0.6422297692938284
F1 score at threshold 0.17 is 0.6471112166735943
F1 score at threshold 0.18 is 0.6500360663621063
F1 score at threshold 0.19 is 0.6531654807516876
F1 score at threshold 0.2 is 0.6565195957604142
F1 score at threshold 0.21 is 0.659221362615423
F1 score at threshold 0.22 is 0.6629596865916846
F1 score at threshold 0.23 is 0.6650047938638542
F1 score at threshold 0.24 is 0.6666236614630371
F1 score at threshold 0.25 is 0.6683126502501463
F1 score at threshold 0.26 is 0.6696785176455183
F1 score at threshold 0.27 is 0.671241010754107
F1 score at threshold 0.28 is 0.6731012447580377
F1 score at threshold 0.29 is 0.6745339949041169
F1 score at threshold 0.3 

In [21]:
total_predictions=lstm.predict(test_input_padded, batch_size=512)
test_labels=list(test_data['target'])
predictions1 = (total_predictions>best_thresh).astype(int)
res=f1_score(test_labels, predictions1[:,0])
print(res)

0.6717309662000682
